In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


In [3]:
Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)
##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'

x = pd.read_csv(allplots_fc).groupby(['Latitude', 'Longitude']).first().reset_index()

fc_all = geemap.pandas_to_ee(x, latitude = "Latitude", longitude = "Longitude") ##FOR DAYMET AND GRIDMET

gce_clip = 'F:/Wetlands shapefiles/GCE domain/GCE_LTER_boundary.shp' ##Now, do the entire GA coast!
gce = geemap.shp_to_ee(gce_clip)

In [4]:
def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

In [5]:
#12/08/22 - adding CALIBRATED FLATS
def addFLATSL7(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(RED-SWIR)/(RED+SWIR) - 41.2*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.972*(RED-SWIR)/(RED+SWIR)-0.008) - 41.2*(0.991*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)-0.0014)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))


def addFLATSL8(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.841*(RED-SWIR)/(RED+SWIR) - 0.019) - 41.2*(0.771*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1) + 0.011)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL9(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(1.225*(RED-SWIR)/(RED+SWIR) + 0.096) - 41.2*(1.038* (NIR-RED)/(NIR+6*RED-7.5*BLUE+1) - 0.004)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.2) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

##ADDING NDVI (for min/max variables)
def addL5ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'GREEN': image.select('SR_B2')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addL8ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

In [6]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

##For Landsat images:
# def rasterExtraction(image):
#     feature = image.sampleRegions(
#         collection = fc_all,
#         scale = 30,
#         tileScale = 16 #ADDED 10/6/2022 - make sure it doesn't affect results (see thread below)
#     )
#     return feature


def rasterExtraction(image, fc_subset):
    return image.sampleRegions(
        collection=fc_subset,  # Use the subset of locations
        scale=30,
        tileScale=8  # Adjust tile scale for performance
    ).map(lambda f: f.set('date', image.date().format('YYYY-MM-dd')))

In [7]:
#Datasets - NOT MASKING FLATS (try again masking later)
ls5_collect = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterBounds(fc_all).map(maskL8sr).map(addFLATSL5)
ls7_collect = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterBounds(fc_all).map(maskL8sr).map(addFLATSL7)
ls8_collect = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(fc_all).map(maskL8sr).map(addFLATSL8)
ls9_collect = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(fc_all).map(maskL8sr).map(addFLATSL8)

#Merge
ultra_col = ls8_collect.merge(ls5_collect).merge(ls7_collect).merge(ls9_collect)

In [16]:
# 1) Map each image to a binary (0/1) mask where flats > 0.2
binary = ultra_col.select('flats') \
    .map(lambda img: img.gte(0.2)    # returns a Boolean image
                    .toFloat()      # convert True→1.0, False→0.0
                    .rename('above'))

# 2) Compute the mean of that 'above' band across the collection
#    mean() here is the same as reduce(ee.Reducer.mean())
fraction = binary.mean().rename('fraction_above')

# 3) Convert to percentage
percent_above = fraction.multiply(1).rename('percent_above')

# 4) Visualize
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['lightgray', 'yellow', 'red']
}

# Create a map and add the layer
m = geemap.Map(center=[37, -95], zoom=4)
m.addLayer(percent_above, vis_params, 'Percent of flats > 0.2')
# m

In [13]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
filename = os.path.join(out_dir, "floodingpercent.tif")

In [11]:
# geemap.ee_export_image(
#     percent_above, filename=filename, scale=30, region=gce.geometry(), file_per_band=False
# )

# geemap.ee_export_image_to_drive(
#     percent_above, description="floodingpercent", folder="export", region=gce.geometry(), scale=30
# )

In [15]:
# ##Export to asset
# export_task = ee.batch.Export.image.toAsset(
#     image=percent_above,
#     description='export_percent_flooding',
#     assetId='projects/boba-430314/assets/percent_flooding',
#     region=gce.geometry(),
#     scale=30,
#     maxPixels=1e13
# )

# export_task.start()

EEException: Permission 'earthengine.assets.create' denied on resource 'projects/boba-430314' (or it may not exist).